### 1. Initialize Network

In [1]:
from random import random

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights' : [random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights' : [random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    
    return network

Test function "initialize_network"

In [2]:
from random import seed

seed(1)
network = initialize_network(2, 1, 2)

for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


### 2. Forward Propagate

#### 2.1 Neuron Activation

Calculate the activation of one neuron given an input

In [3]:
# Calculate neuron pre-activation for an input. We assume that the bias is at the "last" position in the list of weights

# activation = sum(weight_i * input_i) + bias

def activate(weights, inputs):
    atv = weights[-1]  # bias
    for i in range(len(weights)-1):
        atv += weights[i] * inputs[i]
    return atv

#### 2.2 Neuron Transfer

In [4]:
from math import exp

# post-activation (using sigmoid function)

def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

#### 2.3 Forward Propagation

In [5]:
# Forward propagate input to a network output

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
        #print("len(inputs)=", len(inputs))
    
    return inputs

Test forward propagation of our network

In [6]:
network = [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
           [{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)

for layer in network:
    print(layer)

[0.6629970129852887, 0.7253160725279748]
[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748}]


### 3. Back Propagate

#### 3.1 Transfer Derivative

In [7]:
# Calculate the derivative of an neuron output for sigmoid function

def transfer_derivative(output):
    return output * (1.0 - output)

#### 3.2 Error Backpropagation

In [8]:
# Backpropagate error and store in neuros

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network) -1:
            for j in range(len(layer)):
                error = 0.0
                for neuro in network[i+1]:
                    error += (neuro['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [9]:
# test backpropagation of error
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
           [{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]},  {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': 0.011182350443718935}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': -0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': 0.0771723774346327}]


### 4. Train Network

#### 4.1 Update Weights

In [10]:
# Update network weights with error

def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta' ] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

#### 4.2 Train Network

In [11]:
# Train a network for a fixed number of epochs

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i] - outputs[i]) **2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [12]:
# Test training backprop algorithm

seed(1)
dataset = [[2.7810836,2.550537003,0],
           [1.465489372,2.362125076,0],
           [3.396561688,4.400293529,0],
           [1.38807019,1.850220317,0],
           [3.06407232,3.005305973,0],
           [7.627531214,2.759262235,1],
           [5.332441248,2.088626775,1],
           [6.922596716,1.77106367,1],
           [8.675418651,-0.242068655,1],
           [7.673756466,3.508563011,1]]

n_inputs = len(dataset[0]) -1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
    print(layer)

>epoch=0, lrate=0.500, error=6.342
>epoch=1, lrate=0.500, error=5.569
>epoch=2, lrate=0.500, error=5.312
>epoch=3, lrate=0.500, error=5.279
>epoch=4, lrate=0.500, error=5.272
>epoch=5, lrate=0.500, error=5.249
>epoch=6, lrate=0.500, error=5.204
>epoch=7, lrate=0.500, error=5.132
>epoch=8, lrate=0.500, error=5.029
>epoch=9, lrate=0.500, error=4.891
>epoch=10, lrate=0.500, error=4.721
>epoch=11, lrate=0.500, error=4.523
>epoch=12, lrate=0.500, error=4.305
>epoch=13, lrate=0.500, error=4.074
>epoch=14, lrate=0.500, error=3.838
>epoch=15, lrate=0.500, error=3.603
>epoch=16, lrate=0.500, error=3.374
>epoch=17, lrate=0.500, error=3.154
>epoch=18, lrate=0.500, error=2.947
>epoch=19, lrate=0.500, error=2.755
[{'weights': [-0.1719614774454439, 0.8465685728984452, 0.7411689464909673], 'output': 0.917149296364726, 'delta': -0.0003438503139515136}, {'weights': [0.7138725303829672, -0.96351140006207, -0.03939255420495675], 'output': 0.8664364903618547, 'delta': 0.005217401830431264}]
[{'weights': [

### 5. Predict

In [13]:
# Make a prediction with a network

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [14]:
# Test making predictions with the network
dataset = [[1.465489372,2.362125076,0],
           [3.396561688,4.400293529,0],
           [1.38807019,1.850220317,0],
           [3.06407232,3.005305973,0],
           [7.627531214,2.759262235,1],
           [5.332441248,2.088626775,1],
           [6.922596716,1.77106367,1],
           [8.675418651,-0.242068655,1],
           [2.7810836,2.550537003,0],
           [7.673756466,3.508563011,1]]

network = [[{'weights': [-1.482313569067226, 1.8308790073202204, 1.078381922048799]}, {'weights': [0.23244990332399884, 0.3621998343835864, 0.40289821191094327]}],
           [{'weights': [2.5001872433501404, 0.7887233511355132, -1.1026649757805829]}, {'weights': [-2.429350576245497, 0.8357651039198697, 1.0699217181280656]}]]

for row in dataset:
    prediction = predict(network, row)
    print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1


### 6. Wheat Seeds Dataset

In [15]:
from csv import reader
from random import randrange

# Backprop on the Seeds Dataset

# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        #print(row[column].strip())
        row[column] = float(row[column].strip())
        
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row) - 1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
            
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    
    return scores

# Backpropagation Algorithm with Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) -1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return predictions

In [16]:
# Test Backprop on Seeds dataset
seed(1)
# load and prepare data
filename = 'seeds_dataset.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 5
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

>epoch=0, lrate=0.300, error=130.313
>epoch=1, lrate=0.300, error=110.958
>epoch=2, lrate=0.300, error=108.416
>epoch=3, lrate=0.300, error=105.881
>epoch=4, lrate=0.300, error=103.340
>epoch=5, lrate=0.300, error=100.774
>epoch=6, lrate=0.300, error=98.163
>epoch=7, lrate=0.300, error=95.486
>epoch=8, lrate=0.300, error=92.722
>epoch=9, lrate=0.300, error=89.858
>epoch=10, lrate=0.300, error=86.896
>epoch=11, lrate=0.300, error=83.855
>epoch=12, lrate=0.300, error=80.779
>epoch=13, lrate=0.300, error=77.726
>epoch=14, lrate=0.300, error=74.754
>epoch=15, lrate=0.300, error=71.915
>epoch=16, lrate=0.300, error=69.243
>epoch=17, lrate=0.300, error=66.755
>epoch=18, lrate=0.300, error=64.456
>epoch=19, lrate=0.300, error=62.344
>epoch=20, lrate=0.300, error=60.407
>epoch=21, lrate=0.300, error=58.632
>epoch=22, lrate=0.300, error=57.006
>epoch=23, lrate=0.300, error=55.513
>epoch=24, lrate=0.300, error=54.140
>epoch=25, lrate=0.300, error=52.874
>epoch=26, lrate=0.300, error=51.704
>epoc

>epoch=230, lrate=0.300, error=18.842
>epoch=231, lrate=0.300, error=18.818
>epoch=232, lrate=0.300, error=18.791
>epoch=233, lrate=0.300, error=18.763
>epoch=234, lrate=0.300, error=18.736
>epoch=235, lrate=0.300, error=18.710
>epoch=236, lrate=0.300, error=18.686
>epoch=237, lrate=0.300, error=18.665
>epoch=238, lrate=0.300, error=18.649
>epoch=239, lrate=0.300, error=18.636
>epoch=240, lrate=0.300, error=18.626
>epoch=241, lrate=0.300, error=18.620
>epoch=242, lrate=0.300, error=18.617
>epoch=243, lrate=0.300, error=18.615
>epoch=244, lrate=0.300, error=18.613
>epoch=245, lrate=0.300, error=18.611
>epoch=246, lrate=0.300, error=18.606
>epoch=247, lrate=0.300, error=18.597
>epoch=248, lrate=0.300, error=18.584
>epoch=249, lrate=0.300, error=18.566
>epoch=250, lrate=0.300, error=18.541
>epoch=251, lrate=0.300, error=18.511
>epoch=252, lrate=0.300, error=18.474
>epoch=253, lrate=0.300, error=18.432
>epoch=254, lrate=0.300, error=18.384
>epoch=255, lrate=0.300, error=18.331
>epoch=256, 

>epoch=448, lrate=0.300, error=12.724
>epoch=449, lrate=0.300, error=12.712
>epoch=450, lrate=0.300, error=12.700
>epoch=451, lrate=0.300, error=12.688
>epoch=452, lrate=0.300, error=12.676
>epoch=453, lrate=0.300, error=12.664
>epoch=454, lrate=0.300, error=12.653
>epoch=455, lrate=0.300, error=12.641
>epoch=456, lrate=0.300, error=12.629
>epoch=457, lrate=0.300, error=12.618
>epoch=458, lrate=0.300, error=12.607
>epoch=459, lrate=0.300, error=12.595
>epoch=460, lrate=0.300, error=12.584
>epoch=461, lrate=0.300, error=12.573
>epoch=462, lrate=0.300, error=12.562
>epoch=463, lrate=0.300, error=12.551
>epoch=464, lrate=0.300, error=12.539
>epoch=465, lrate=0.300, error=12.529
>epoch=466, lrate=0.300, error=12.518
>epoch=467, lrate=0.300, error=12.507
>epoch=468, lrate=0.300, error=12.496
>epoch=469, lrate=0.300, error=12.485
>epoch=470, lrate=0.300, error=12.475
>epoch=471, lrate=0.300, error=12.464
>epoch=472, lrate=0.300, error=12.453
>epoch=473, lrate=0.300, error=12.443
>epoch=474, 

>epoch=172, lrate=0.300, error=17.758
>epoch=173, lrate=0.300, error=17.660
>epoch=174, lrate=0.300, error=17.563
>epoch=175, lrate=0.300, error=17.469
>epoch=176, lrate=0.300, error=17.376
>epoch=177, lrate=0.300, error=17.286
>epoch=178, lrate=0.300, error=17.197
>epoch=179, lrate=0.300, error=17.110
>epoch=180, lrate=0.300, error=17.025
>epoch=181, lrate=0.300, error=16.942
>epoch=182, lrate=0.300, error=16.861
>epoch=183, lrate=0.300, error=16.781
>epoch=184, lrate=0.300, error=16.703
>epoch=185, lrate=0.300, error=16.627
>epoch=186, lrate=0.300, error=16.552
>epoch=187, lrate=0.300, error=16.479
>epoch=188, lrate=0.300, error=16.408
>epoch=189, lrate=0.300, error=16.339
>epoch=190, lrate=0.300, error=16.271
>epoch=191, lrate=0.300, error=16.204
>epoch=192, lrate=0.300, error=16.139
>epoch=193, lrate=0.300, error=16.076
>epoch=194, lrate=0.300, error=16.014
>epoch=195, lrate=0.300, error=15.954
>epoch=196, lrate=0.300, error=15.896
>epoch=197, lrate=0.300, error=15.839
>epoch=198, 

>epoch=388, lrate=0.300, error=12.542
>epoch=389, lrate=0.300, error=12.502
>epoch=390, lrate=0.300, error=12.462
>epoch=391, lrate=0.300, error=12.423
>epoch=392, lrate=0.300, error=12.384
>epoch=393, lrate=0.300, error=12.346
>epoch=394, lrate=0.300, error=12.309
>epoch=395, lrate=0.300, error=12.272
>epoch=396, lrate=0.300, error=12.235
>epoch=397, lrate=0.300, error=12.199
>epoch=398, lrate=0.300, error=12.164
>epoch=399, lrate=0.300, error=12.129
>epoch=400, lrate=0.300, error=12.095
>epoch=401, lrate=0.300, error=12.061
>epoch=402, lrate=0.300, error=12.028
>epoch=403, lrate=0.300, error=11.995
>epoch=404, lrate=0.300, error=11.962
>epoch=405, lrate=0.300, error=11.931
>epoch=406, lrate=0.300, error=11.899
>epoch=407, lrate=0.300, error=11.869
>epoch=408, lrate=0.300, error=11.839
>epoch=409, lrate=0.300, error=11.809
>epoch=410, lrate=0.300, error=11.780
>epoch=411, lrate=0.300, error=11.751
>epoch=412, lrate=0.300, error=11.723
>epoch=413, lrate=0.300, error=11.695
>epoch=414, 

...

>epoch=371, lrate=0.300, error=35.527
>epoch=372, lrate=0.300, error=34.935
>epoch=373, lrate=0.300, error=34.355
>epoch=374, lrate=0.300, error=33.786
>epoch=375, lrate=0.300, error=33.229
>epoch=376, lrate=0.300, error=32.682
>epoch=377, lrate=0.300, error=32.145
>epoch=378, lrate=0.300, error=31.617
>epoch=379, lrate=0.300, error=31.098
>epoch=380, lrate=0.300, error=30.588
>epoch=381, lrate=0.300, error=30.085
>epoch=382, lrate=0.300, error=29.590
>epoch=383, lrate=0.300, error=29.103
>epoch=384, lrate=0.300, error=28.623
>epoch=385, lrate=0.300, error=28.150
>epoch=386, lrate=0.300, error=27.685
>epoch=387, lrate=0.300, error=27.227
>epoch=388, lrate=0.300, error=26.775
>epoch=389, lrate=0.300, error=26.332
>epoch=390, lrate=0.300, error=25.895
>epoch=391, lrate=0.300, error=25.466
>epoch=392, lrate=0.300, error=25.044
>epoch=393, lrate=0.300, error=24.631
>epoch=394, lrate=0.300, error=24.225
>epoch=395, lrate=0.300, error=23.828
>epoch=396, lrate=0.300, error=23.441
>epoch=397, 

>epoch=99, lrate=0.300, error=36.443
>epoch=100, lrate=0.300, error=36.393
>epoch=101, lrate=0.300, error=36.344
>epoch=102, lrate=0.300, error=36.296
>epoch=103, lrate=0.300, error=36.249
>epoch=104, lrate=0.300, error=36.201
>epoch=105, lrate=0.300, error=36.155
>epoch=106, lrate=0.300, error=36.109
>epoch=107, lrate=0.300, error=36.063
>epoch=108, lrate=0.300, error=36.018
>epoch=109, lrate=0.300, error=35.973
>epoch=110, lrate=0.300, error=35.928
>epoch=111, lrate=0.300, error=35.884
>epoch=112, lrate=0.300, error=35.840
>epoch=113, lrate=0.300, error=35.797
>epoch=114, lrate=0.300, error=35.754
>epoch=115, lrate=0.300, error=35.711
>epoch=116, lrate=0.300, error=35.668
>epoch=117, lrate=0.300, error=35.626
>epoch=118, lrate=0.300, error=35.584
>epoch=119, lrate=0.300, error=35.542
>epoch=120, lrate=0.300, error=35.500
>epoch=121, lrate=0.300, error=35.459
>epoch=122, lrate=0.300, error=35.418
>epoch=123, lrate=0.300, error=35.377
>epoch=124, lrate=0.300, error=35.337
>epoch=125, l

>epoch=324, lrate=0.300, error=21.826
>epoch=325, lrate=0.300, error=21.662
>epoch=326, lrate=0.300, error=21.501
>epoch=327, lrate=0.300, error=21.344
>epoch=328, lrate=0.300, error=21.189
>epoch=329, lrate=0.300, error=21.037
>epoch=330, lrate=0.300, error=20.888
>epoch=331, lrate=0.300, error=20.742
>epoch=332, lrate=0.300, error=20.599
>epoch=333, lrate=0.300, error=20.459
>epoch=334, lrate=0.300, error=20.321
>epoch=335, lrate=0.300, error=20.187
>epoch=336, lrate=0.300, error=20.055
>epoch=337, lrate=0.300, error=19.926
>epoch=338, lrate=0.300, error=19.799
>epoch=339, lrate=0.300, error=19.676
>epoch=340, lrate=0.300, error=19.555
>epoch=341, lrate=0.300, error=19.437
>epoch=342, lrate=0.300, error=19.321
>epoch=343, lrate=0.300, error=19.209
>epoch=344, lrate=0.300, error=19.099
>epoch=345, lrate=0.300, error=18.992
>epoch=346, lrate=0.300, error=18.887
>epoch=347, lrate=0.300, error=18.786
>epoch=348, lrate=0.300, error=18.687
>epoch=349, lrate=0.300, error=18.591
>epoch=350, 

>epoch=67, lrate=0.300, error=33.976
>epoch=68, lrate=0.300, error=33.636
>epoch=69, lrate=0.300, error=33.302
>epoch=70, lrate=0.300, error=32.975
>epoch=71, lrate=0.300, error=32.655
>epoch=72, lrate=0.300, error=32.342
>epoch=73, lrate=0.300, error=32.036
>epoch=74, lrate=0.300, error=31.736
>epoch=75, lrate=0.300, error=31.443
>epoch=76, lrate=0.300, error=31.157
>epoch=77, lrate=0.300, error=30.877
>epoch=78, lrate=0.300, error=30.603
>epoch=79, lrate=0.300, error=30.336
>epoch=80, lrate=0.300, error=30.075
>epoch=81, lrate=0.300, error=29.820
>epoch=82, lrate=0.300, error=29.571
>epoch=83, lrate=0.300, error=29.328
>epoch=84, lrate=0.300, error=29.091
>epoch=85, lrate=0.300, error=28.860
>epoch=86, lrate=0.300, error=28.634
>epoch=87, lrate=0.300, error=28.414
>epoch=88, lrate=0.300, error=28.200
>epoch=89, lrate=0.300, error=27.991
>epoch=90, lrate=0.300, error=27.787
>epoch=91, lrate=0.300, error=27.588
>epoch=92, lrate=0.300, error=27.394
>epoch=93, lrate=0.300, error=27.205
>

>epoch=294, lrate=0.300, error=16.741
>epoch=295, lrate=0.300, error=16.683
>epoch=296, lrate=0.300, error=16.625
>epoch=297, lrate=0.300, error=16.568
>epoch=298, lrate=0.300, error=16.511
>epoch=299, lrate=0.300, error=16.453
>epoch=300, lrate=0.300, error=16.397
>epoch=301, lrate=0.300, error=16.340
>epoch=302, lrate=0.300, error=16.283
>epoch=303, lrate=0.300, error=16.227
>epoch=304, lrate=0.300, error=16.171
>epoch=305, lrate=0.300, error=16.115
>epoch=306, lrate=0.300, error=16.059
>epoch=307, lrate=0.300, error=16.004
>epoch=308, lrate=0.300, error=15.948
>epoch=309, lrate=0.300, error=15.893
>epoch=310, lrate=0.300, error=15.838
>epoch=311, lrate=0.300, error=15.783
>epoch=312, lrate=0.300, error=15.729
>epoch=313, lrate=0.300, error=15.674
>epoch=314, lrate=0.300, error=15.620
>epoch=315, lrate=0.300, error=15.566
>epoch=316, lrate=0.300, error=15.512
>epoch=317, lrate=0.300, error=15.458
>epoch=318, lrate=0.300, error=15.405
>epoch=319, lrate=0.300, error=15.352
>epoch=320, 